In [1]:
!pip install pulp
from pulp import *
import pandas as pd

# Define crop data
crops = {
    "rice": {"temperature": (20, 35), "pH": (5.5, 7), "net_return": 100},
    "wheat": {"temperature": (15, 30), "pH": (6, 7.5), "net_return": 120},
    "coffee": {"temperature": (25, 35), "pH": (6, 7.5), "net_return": 200},
    "maize": {"temperature": (22, 30), "pH": (5.8, 7.2), "net_return": 150},
    "beans": {"temperature": (20, 28), "pH": (6, 7), "net_return": 180},
    "sugarcane": {"temperature": (25, 35), "pH": (5.5, 7), "net_return": 250},
    "oat": {"temperature": (15, 20), "pH": (6, 7), "net_return": 90},
    "berseem": {"temperature": (18, 25), "pH": (6, 7.5), "net_return": 110},
}

# Input temperature and pH
temperature = float(input("Enter temperature: "))
pH = float(input("Enter pH: "))

# Filter crops based on temperature and pH
suitable_crops = [crop for crop, data in crops.items() if data["temperature"][0] <= temperature <= data["temperature"][1]
                  and data["pH"][0] <= pH <= data["pH"][1]]

print("Crops suitable for the given temperature and pH:")
print(pd.Series(suitable_crops))

# Initialize optimization model
model = LpProblem("CropRotationOptimization", LpMaximize)

# Define decision variables
rotation_vars = LpVariable.dicts("Rotation", (suitable_crops, range(1, 4), range(1, 5)), 0, cat="Binary")

# Define objective function: Maximize profit
model += lpSum(rotation_vars[crop][year][season] * crops[crop]["net_return"]
               for crop in suitable_crops for year in range(1, 4) for season in range(1, 5))

# Constraint: Only one crop can be planted in each season
for year in range(1, 4):
    for season in range(1, 5):
        model += lpSum(rotation_vars[crop][year][season] for crop in suitable_crops) == 1

# Constraint: One crop cannot be planted in two consecutive seasons
for crop in suitable_crops:
    for year in range(1, 4):
        for season in range(1, 4):
            model += rotation_vars[crop][year][season] + rotation_vars[crop][year][season + 1] <= 1

# Constraint: Two crops from the same family cannot be grown consecutively
family_dict = {
    "rice": "grain",
    "wheat": "grain",
    "oat": "grain",
    "maize": "grain",
    "beans": "legume",
    "sugarcane": "cane",
    "coffee": "bean",
    "berseem": "legume"
}

for year in range(1, 4):
    for season in range(1, 4):
        for crop in suitable_crops:
            if season < 4:  # Exclude the last season of each year from this constraint
                family = family_dict[crop]
                next_crop = [c for c in suitable_crops if family_dict[c] == family]
                if len(next_crop) >= 2:
                    model += lpSum(rotation_vars[crop][year][season] + rotation_vars[next_crop[0]][year][season + 1] +
                                   rotation_vars[next_crop[1]][year][season + 1]) <= 2
                elif len(next_crop) == 1:
                    model += lpSum(rotation_vars[crop][year][season] + rotation_vars[next_crop[0]][year][season + 1]) <= 1

# Constraint: At least one leguminous plant should be grown in one year
for year in range(1, 4):
    legumes = [crop for crop in suitable_crops if family_dict[crop] == "legume"]
    model += lpSum(rotation_vars[crop][year][season] for crop in legumes for season in range(1, 5)) >= 1

# Solve the model
model.solve()

# Print the optimized rotation schedule and objective value
print("\nOptimal Rotation Schedule:")
for year in range(1, 4):
    print(f"\nYear {year}:")
    for season in range(1, 5):
        for crop in suitable_crops:
            if rotation_vars[crop][year][season].varValue == 1:
                print(f"Season {season}: {crop}")

print("\nObjective Value (Maximum Profit):", value(model.objective))

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 29.2 MB/s eta 0:00:00
Enter temperature: 30
Enter pH: 7
Crops suitable for the given temperature and pH:
0         rice
1        wheat
2       coffee
3        maize
4    sugarcane
dtype: object

Optimal Rotation Schedule:

Year 1:
Season 1: sugarcane
Season 2: coffee
Season 3: sugarcane
Season 4: sugarcane

Year 2:
Season 1: sugarcane
Season 2: sugarcane
Season 3: coffee
Season 4: sugarcane

Year 3:
Season 1: sugarcane
Season 2: coffee
Season 3: sugarcane
Season 4: sugarcane

Objective Value (Maximum Profit): 2850.0
